In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["Date"]= dt.datetime.today()

In [6]:
# storing the rate of growth for analysis

#importing previous data
growth_last = pd.read_csv("growth.csv")
# #staging new data to add
growth_df = pd.DataFrame()


growth_df["Date"]=[0]
growth_df["NewCases"]=[0]
growth_df["TotalCases"] =[0]
growth_df["VarianceMultiple"]=[0]
growth_df["GrowthFactor"]=[0]

growth_df["Date"]= dt.datetime.today()
growth_df["NewCases"] = corona_df["NewCases"].sum()
growth_df["TotalCases"] =corona_df["TotalCases"].sum()
growth_df["VarianceMultiple"]= 1 + (growth_df["NewCases"]/growth_df["TotalCases"])
growth_df["GrowthFactor"]=[growth_df["NewCases"][0]/growth_last["NewCases"][len(growth_last)-1]]

#appending new data
    
updated_growth_df = growth_last.append(growth_df)
updated_growth_df.reset_index(inplace=True,drop=True)
updated_growth_df.to_csv('growth.csv',index = False)
updated_growth_df.to_html('growth.html',index = False)
updated_growth_df.to_json('growth.json',orient = "records")
updated_growth_df

,Date,NewCases,TotalCases,VarianceMultiple,GrowthFactor
0,2020-03-11 00:57:05.757569,297.0,119220,1.002491,1.000000
1,2020-03-11 01:03:51.872781,297.0,119220,1.002491,1.000000
2,2020-03-11 01:16:11.835765,302.0,119225,1.002533,1.016835
3,2020-03-11 20:18:49.306369,7113.0,126061,1.056425,23.552980
4,2020-03-11 20:38:41.539226,7113.0,126061,1.056425,1.000000
5,2020-03-12 00:06:55.727048,178.0,126367,1.001409,0.025025
6,2020-03-13 19:58:32.715439,10783.0,145360,1.074181,60.578652
7,2020-03-13 20:00:07.275754,10783.0,145360,1.074181,1.000000
8,2020-03-13 21:43:27.476069,11051.0,145628,1.075885,1.024854
9,2020-03-14 01:31:08.811147,201.0,145673,1.001380,0.018188


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)

#add time stamp


df_country_list = df_used["Country"].tolist()
df_used
#df_country_list

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Date,Longitude,Latitude
0,China,80881,21.0,3226.0,13.0,68715.0,8940,3226.0,56.2,2020-03-17 12:07:19.800204,103.819074,36.561765
1,Italy,27980,0.0,2158.0,0.0,2749.0,23073,1851.0,0.0,2020-03-17 12:07:19.800204,12.070013,42.796626
2,Iran,16169,1178.0,988.0,135.0,5389.0,9792,0.0,192.5,2020-03-17 12:07:19.800204,54.274070,32.575033
3,Spain,11409,1467.0,510.0,168.0,1028.0,9871,563.0,244.0,2020-03-17 12:07:19.800204,-3.647550,40.244487
4,Germany,8588,1316.0,23.0,6.0,67.0,8498,2.0,102.5,2020-03-17 12:07:19.800204,10.385781,51.106982
...,...,...,...,...,...,...,...,...,...,...,...,...
136,Mauritania,1,0.0,0.0,0.0,0.0,1,0.0,0.2,2020-03-17 12:07:19.800204,-10.347798,20.257367
137,Somalia,1,0.0,0.0,0.0,0.0,1,0.0,0.1,2020-03-17 12:07:19.800204,45.707145,4.750629
138,Suriname,1,0.0,0.0,0.0,0.0,1,0.0,0.0,2020-03-17 12:07:19.800204,-55.912346,4.130554
139,Tanzania,1,0.0,0.0,0.0,0.0,1,0.0,0.0,2020-03-17 12:07:19.800204,34.813100,-6.275654


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"China","TotalCases":80881,"NewCases":21.0,"TotalDeaths":3226.0,"NewDeaths":13.0,"TotalRecovered":68715.0,"ActiveCases":8940,"CriticalCases":3226.0,"Tot\\u00a0Cases\\/1M pop":56.2,"Date":1584446839800,"Longitude":103.8190735,"Latitude":36.56176546},{"Country":"Italy","TotalCases":27980,"NewCases":0.0,"TotalDeaths":2158.0,"NewDeaths":0.0,"TotalRecovered":2749.0,"ActiveCases":23073,"CriticalCases":1851.0,"Tot\\u00a0Cases\\/1M pop":0.0,"Date":1584446839800,"Longitude":12.07001339,"Latitude":42.79662641},{"Country":"Iran","TotalCases":16169,"NewCases":1178.0,"TotalDeaths":988.0,"NewDeaths":135.0,"TotalRecovered":5389.0,"ActiveCases":9792,"CriticalCases":0.0,"Tot\\u00a0Cases\\/1M pop":192.5,"Date":1584446839800,"Longitude":54.27407004,"Latitude":32.57503292},{"Country":"Spain","TotalCases":11409,"NewCases":1467.0,"TotalDeaths":510.0,"NewDeaths":168.0,"TotalRecovered":1028.0,"ActiveCases":9871,"CriticalCases":563.0,"Tot\\u00a0Cases\\/1M pop":244.0,"Date":1584446839800,"Longitude